# APF

In [ ]:
%matplotlib inline

In [ ]:
# === PARAMETERS ===
NUM_AGENTS = 500
GOAL_TOL = 0.001
STEP_SIZE = 0.05
K_ATTR = 1.0      # Attractive potential gain
K_REP = 0.0002    # Repulsive potential gain
REPULSION_RADIUS = 0.55
MAX_REP_FORCE = 1.0
MIN_DIST = 0.05   # Prevent division by very small values
MAX_STEP = 0.07   # Clamp maximum step per agent
DAMPING = 0.7     # Optional: smooth movement

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# === INITIAL CONDITIONS ===
np.random.seed(0)
positions = np.random.rand(NUM_AGENTS, 2) * np.array((-2, 1))

# === RANDOM GAUSSIAN GOALS ===
# goals = np.tile(np.array([1, 0]), (NUM_AGENTS, 1) )
goal_mean = np.array([1.0, 1.0])
goal_cov = np.array([[0.05, 0], [0, 1]])  # Small spread
goals = np.random.multivariate_normal(mean=goal_mean, cov=goal_cov, size=NUM_AGENTS)

# === APF PLANNING FUNCTION ===
def compute_apf_forces(pos, goals):
    forces = np.zeros_like(pos)
    for i in range(len(pos)):
        # Attractive force
        diff = goals[i] - pos[i]
        f_attr = K_ATTR * diff

        # Repulsive force from other agents
        f_rep = np.zeros(2)
        for j in range(len(pos)):
            if i == j:
                continue
            diff_ij = pos[i] - pos[j]
            dist = np.linalg.norm(diff_ij)
            if dist < REPULSION_RADIUS and dist > 1e-6:
                # Clamp small distances to avoid singularity
                dist_clamped = max(dist, MIN_DIST)
                rep_mag = K_REP * (1.0 / dist_clamped - 1.0 / REPULSION_RADIUS) / (dist_clamped**2)
                rep_mag = np.clip(rep_mag, 0, MAX_REP_FORCE)
                f_rep += rep_mag * (diff_ij / dist_clamped)

        forces[i] = f_attr + f_rep
    return forces

# === VISUALIZATION SETUP ===
fig, ax = plt.subplots()
scat = ax.scatter([], [], c='blue')
goal_plot = ax.scatter(goals[:, 0], goals[:, 1], c='green', marker='x')
ax.set_xlim(-1, 3)
ax.set_ylim(-1, 3)
ax.set_aspect('equal')

# === SIMULATION LOOP ===
stopped = False

def update(frame):
    global positions, stopped

    if not stopped:
        dists_to_goals = np.linalg.norm(positions - goals, axis=1)
        if np.any(dists_to_goals < GOAL_TOL):
            stopped = True
        else:
            forces = compute_apf_forces(positions, goals)
            for i in range(len(positions)):
                step = STEP_SIZE * forces[i]
                norm = np.linalg.norm(step)
                if norm > MAX_STEP:
                    step = MAX_STEP * step / norm
                positions[i] += DAMPING * step  # Apply damping

    scat.set_offsets(positions)
    return scat,

ani = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True)
plt.title("Multi-Agent APF with Stability Control")

from IPython.display import HTML
HTML(ani.to_jshtml())